In [3]:
import pandas as pd

# Load the CSV file to inspect the data
file_path = 'data/Human_Evaluation.csv'
data = pd.read_csv(file_path)

# Display the first few rows and the structure of the data
data.head()

,Question_1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Question_2,Unnamed: 5,Unnamed: 6,Unnamed: 7,Question_3,Unnamed: 9,...,Unnamed: 30,Unnamed: 31,Question_9,Unnamed: 33,Unnamed: 34,Unnamed: 35,Question_10,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Model_A,Model_B,Model_C,Model_D,Model_A,Model_B,Model_C,Model_D,Model_A,Model_B,...,Model_C,Model_D,Model_A,Model_B,Model_C,Model_D,Model_A,Model_B,Model_C,Model_D
1,4th (Bad),3rd,2nd,1st (Best),4th (Bad),1st (Best),3rd,2nd,2nd,1st (Best),...,4th (Bad),2nd,1st (Best),4th (Bad),2nd,3rd,1st (Best),4th (Bad),3rd,2nd
2,3rd,1st (Best),2nd,4th (Bad),4th (Bad),2nd,3rd,1st (Best),4th (Bad),3rd,...,2nd,1st (Best),4th (Bad),1st (Best),3rd,2nd,4th (Bad),1st (Best),3rd,2nd
3,4th (Bad),3rd,2nd,1st (Best),4th (Bad),2nd,3rd,1st (Best),4th (Bad),3rd,...,4th (Bad),3rd,2nd,4th (Bad),3rd,1st (Best),1st (Best),4th (Bad),3rd,2nd
4,4th (Bad),2nd,3rd,1st (Best),4th (Bad),2nd,3rd,1st (Best),4th (Bad),2nd,...,4th (Bad),3rd,2nd,4th (Bad),3rd,1st (Best),4th (Bad),3rd,2nd,1st (Best)


In [15]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:0000:01


In [43]:
# Strip the data frame from any redundant columns and rows (like headers repeated)
data_cleaned = data.iloc[1:, :].copy()  # Ignore the first row which repeats the model names

# Rename columns based on the content: each question block has 4 columns for 4 models
columns = []
for i in range(1, 11):  # Assuming there are 10 questions from the data head
    for model in ['Model_A', 'Model_B', 'Model_C', 'Model_D']:
        columns.append(f"Question_{i}_{model}")

data_cleaned.columns = columns

# Replace ordinal rankings with numeric scores
rank_mapping = {'1st (Best)': 1, '2nd': 2, '3rd': 3, '4th (Bad)': 4}
data_cleaned.replace(rank_mapping, inplace=True)

# Display the cleaned data to verify transformations
data_cleaned = data_cleaned.dropna()

data_cleaned.insert(0, "humen number", [1, 2, 3, 4, 5], True)

data_cleaned

,humen number,Question_1_Model_A,Question_1_Model_B,Question_1_Model_C,Question_1_Model_D,Question_2_Model_A,Question_2_Model_B,Question_2_Model_C,Question_2_Model_D,Question_3_Model_A,...,Question_8_Model_C,Question_8_Model_D,Question_9_Model_A,Question_9_Model_B,Question_9_Model_C,Question_9_Model_D,Question_10_Model_A,Question_10_Model_B,Question_10_Model_C,Question_10_Model_D
1,1,1.0,2.0,3.0,4.0,1.0,4.0,2.0,3.0,3.0,...,1.0,3.0,4.0,1.0,3.0,2.0,4.0,1.0,2.0,3.0
2,2,2.0,4.0,3.0,1.0,1.0,3.0,2.0,4.0,1.0,...,3.0,4.0,1.0,4.0,2.0,3.0,1.0,4.0,2.0,3.0
3,3,1.0,2.0,3.0,4.0,1.0,3.0,2.0,4.0,1.0,...,1.0,2.0,3.0,1.0,2.0,4.0,4.0,1.0,2.0,3.0
4,4,1.0,3.0,2.0,4.0,1.0,3.0,2.0,4.0,1.0,...,1.0,2.0,3.0,1.0,2.0,4.0,1.0,2.0,3.0,4.0
5,5,1.0,2.0,3.0,4.0,1.0,2.0,3.0,4.0,1.0,...,3.0,4.0,1.0,2.0,3.0,4.0,1.0,2.0,3.0,4.0


In [44]:
data_cleaned.to_csv('data/Human_Evaluation_Cleaned.csv', index=False)

In [59]:
from sklearn.preprocessing import LabelEncoder

# Reshape the data from wide to long format
long_format = pd.melt(data_cleaned, id_vars=['humen number'], var_name='question_model', value_name='rank')

# Split the 'question_model' column into separate 'question' and 'model' columns
long_format[['question', 'model']] = long_format['question_model'].str.rsplit('_', n=1, expand=True)
long_format.drop(columns='question_model', inplace=True)

# Convert 'question' and 'model' to categorical codes
label_encoder_question = LabelEncoder()
label_encoder_model = LabelEncoder()
long_format['question'] = label_encoder_question.fit_transform(long_format['question'])
long_format['model'] = label_encoder_model.fit_transform(long_format['model'])

# View the restructured data
# long_format.to_csv('data/Human_Evaluation_Long.csv', index=False)
long_format

,humen number,rank,question,model
0,1,1.0,1,0
1,2,2.0,1,0
2,3,1.0,1,0
3,4,1.0,1,0
4,5,1.0,1,0
...,...,...,...,...
195,1,3.0,0,3
196,2,3.0,0,3
197,3,3.0,0,3
198,4,4.0,0,3


In [61]:
import pandas as pd
import pingouin as pg

# Load your data
data = pd.read_csv('data/Human_Evaluation_Long.csv')

# Print current column names to verify
print("Current column names:", data.columns)

# Ensure column names are correctly set (adjust as necessary based on the output above)
data.columns = ['rater', 'rating', 'question', 'model']

# Calculate ICC
icc_results = pg.intraclass_corr(data=data, targets='question', raters='rater', ratings='rating', nan_policy='omit')
print(icc_results)


Current column names: Index(['humen number', 'rank', 'question', 'model'], dtype='object')
    Type              Description  ICC   F  df1  df2  pval       CI95%
0   ICC1   Single raters absolute  NaN NaN    9   40   NaN  [nan, nan]
1   ICC2     Single random raters  NaN NaN    9   36   NaN  [nan, nan]
2   ICC3      Single fixed raters  NaN NaN    9   36   NaN  [nan, nan]
3  ICC1k  Average raters absolute  NaN NaN    9   40   NaN  [nan, nan]
4  ICC2k    Average random raters  NaN NaN    9   36   NaN  [nan, nan]
5  ICC3k     Average fixed raters  NaN NaN    9   36   NaN  [nan, nan]


/Users/tongfah/anaconda3/envs/nlp/lib/python3.11/site-packages/pingouin/reliability.py:312: RuntimeWarning: invalid value encountered in scalar divide
  icc1 = (msb - msw) / (msb + (k - 1) * msw)
/Users/tongfah/anaconda3/envs/nlp/lib/python3.11/site-packages/pingouin/reliability.py:313: RuntimeWarning: invalid value encountered in scalar divide
  icc2 = (msb - mse) / (msb + (k - 1) * mse + k * (msj - mse) / n)
/Users/tongfah/anaconda3/envs/nlp/lib/python3.11/site-packages/pingouin/reliability.py:314: RuntimeWarning: invalid value encountered in scalar divide
  icc3 = (msb - mse) / (msb + (k - 1) * mse)
/Users/tongfah/anaconda3/envs/nlp/lib/python3.11/site-packages/pingouin/reliability.py:315: RuntimeWarning: invalid value encountered in scalar divide
  icc1k = (msb - msw) / msb
/Users/tongfah/anaconda3/envs/nlp/lib/python3.11/site-packages/pingouin/reliability.py:316: RuntimeWarning: invalid value encountered in scalar divide
  icc2k = (msb - mse) / (msb + (msj - mse) / n)
/Users/tongf